## Healthcare Chatbot

In [1]:
!cat /proc/meminfo | grep Mem

MemTotal:       13286964 kB
MemFree:         9375316 kB
MemAvailable:   12351692 kB


## Repo set up and startup details

In [2]:
import os

# Go to base
%cd /content

# Clone repo if it doesn't exist in this runtime
if not os.path.exists("Healthcare-Chatbot"):
    !git clone https://github.com/ryrynbob/Healthcare-Chatbot.git

# Move into repo
%cd Healthcare-Chatbot

print("CWD:", os.getcwd())
print("Files:", os.listdir())


/content
Cloning into 'Healthcare-Chatbot'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 7), reused 2 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 20.63 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/Healthcare-Chatbot
CWD: /content/Healthcare-Chatbot
Files: ['README.md', '.git', 'app.py', 'ryan_nguyen_chatbot_py.py', 'healthcare_data.json']


## Install Dependencies

In [3]:
!pip install sentence-transformers faiss-cpu transformers torch numpy gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 75.5 MB/s eta 0:00:00


In [4]:
# Or, if already cloned, pull latest changes
!git pull


Already up to date.


In [5]:
# 5. Verify the connection and files
print("\nFiles in your project folder:")
!ls


Files in your project folder:
app.py	healthcare_data.json  README.md  ryan_nguyen_chatbot_py.py


In [6]:
print("Colab is alive")


Colab is alive


In [7]:
import os

# Go to base directory
%cd /content

# Clone repo only if not already here in this runtime
if not os.path.exists("Healthcare-Chatbot"):
    !git clone https://github.com/ryrynbob/Healthcare-Chatbot.git

# Move into the repo
%cd Healthcare-Chatbot

print("CWD:", os.getcwd())
print("Files:", os.listdir())


/content
/content/Healthcare-Chatbot
CWD: /content/Healthcare-Chatbot
Files: ['README.md', '.git', 'app.py', 'ryan_nguyen_chatbot_py.py', 'healthcare_data.json']


## Load Knowledge BASE

In [8]:
import json

KB_FILE = "healthcare_data.json"

with open(KB_FILE, "r", encoding="utf-8") as f:
    KB = json.load(f)

KB_QUESTIONS = [item["question"] for item in KB]
KB_ANSWERS = [item["answer"] for item in KB]

print("Loaded KB entries:", len(KB))
print("First entry:", KB[0])


Loaded KB entries: 50
First entry: {'id': 'faq_01', 'question': 'What is the cardiovascular system?', 'answer': 'The cardiovascular system includes your heart and blood vessels. Its main job is to pump blood, delivering oxygen and nutrients to your body and carrying away waste products. Keeping it healthy lowers the risk of heart disease and stroke.'}


# Load Embedding Model and Build FAISS Index

In [9]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# 1. Load the embedding model
EMBEDDER = SentenceTransformer(EMBEDDING_MODEL_NAME)
print("Loaded embedding model:", EMBEDDING_MODEL_NAME)

# 2. Encode all questions
question_embeddings = EMBEDDER.encode(
    KB_QUESTIONS,
    convert_to_numpy=True,
    show_progress_bar=False
).astype("float32")

print("Embeddings shape:", question_embeddings.shape)

# 3. Normalize for cosine similarity
faiss.normalize_L2(question_embeddings)

# 4. Build FAISS index
dim = question_embeddings.shape[1]
INDEX = faiss.IndexFlatIP(dim)
INDEX.add(question_embeddings)

print("FAISS index size:", INDEX.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2
Embeddings shape: (50, 384)
FAISS index size: 50


## Retrieval Helpers

In [10]:
def retrieve_answers(query, k=3):
    """
    Given a user query string, return top-k closest Q&A from the KB.
    """
    # Encode query
    query_emb = EMBEDDER.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(query_emb)

    # Search FAISS index
    distances, indices = INDEX.search(query_emb, k)

    results = []
    for score, idx in zip(distances[0], indices[0]):
        results.append({
            "id": KB[idx]["id"],
            "kb_question": KB[idx]["question"],
            "kb_answer": KB[idx]["answer"],
            "similarity": float(score)
        })
    return results

def answer_question(query, k=3):
    """
    Uses the retriever to show the best match for the user's query.
    """
    matches = retrieve_answers(query, k=k)
    best = matches[0]
    print("User:", query)
    print("Matched FAQ:", best["kb_question"])
    print(f"Similarity: {best['similarity']:.3f}\n")
    print("Answer:")
    print(best["kb_answer"])
    return best


## Test

In [11]:
answer_question("What do beta blockers do?")


User: What do beta blockers do?
Matched FAQ: What are beta-blockers and how do they work?
Similarity: 0.953

Answer:
Beta-blockers are a class of heart medication that reduce the effects of stress hormones like adrenaline. They slow the heart rate and lower blood pressure, which helps the heart work more efficiently. They are often used for conditions such as high blood pressure, certain heart rhythm problems, and heart failure. Only a doctor can decide if they are appropriate for you.


{'id': 'faq_05',
 'kb_question': 'What are beta-blockers and how do they work?',
 'kb_answer': 'Beta-blockers are a class of heart medication that reduce the effects of stress hormones like adrenaline. They slow the heart rate and lower blood pressure, which helps the heart work more efficiently. They are often used for conditions such as high blood pressure, certain heart rhythm problems, and heart failure. Only a doctor can decide if they are appropriate for you.',
 'similarity': 0.9533159732818604}

In [12]:
answer_question("What are signs of a heart attack?")
answer_question("What is heart failure?")


User: What are signs of a heart attack?
Matched FAQ: What are common symptoms of a heart attack?
Similarity: 0.965

Answer:
Symptoms can include chest discomfort or pain that may feel like pressure, squeezing, or fullness; pain in the arm, jaw, neck, back, or stomach; shortness of breath; nausea; cold sweat; or lightheadedness. Symptoms can differ between people. If you think you or someone else might be having a heart attack, call emergency services immediately.
User: What is heart failure?
Matched FAQ: What is heart failure?
Similarity: 1.000

Answer:
Heart failure means the heart is not pumping blood as well as it should. It does not mean the heart has stopped, but that it struggles to meet the body's needs. Symptoms can include shortness of breath, swelling in the legs, and fatigue. Only a healthcare provider can diagnose heart failure using your symptoms, exam, and tests.


{'id': 'faq_23',
 'kb_question': 'What is heart failure?',
 'kb_answer': "Heart failure means the heart is not pumping blood as well as it should. It does not mean the heart has stopped, but that it struggles to meet the body's needs. Symptoms can include shortness of breath, swelling in the legs, and fatigue. Only a healthcare provider can diagnose heart failure using your symptoms, exam, and tests.",
 'similarity': 1.0}

## Implement Retrieval Logic + Similarity Threshold

In [13]:
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr


In [14]:
# Config and Globals

# --- Configuration ---
KB_FILE = "healthcare_data.json"
INDEX_FILE = "faq.index"

EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
GENERATOR_MODEL_NAME = "google/flan-t5-base"   # if this crashes, change to flan-t5-small

SIMILARITY_THRESHOLD = 0.6   # you can tune this later

# Globals
KB = []
KB_QUESTIONS = []
KB_ANSWERS = []
EMBEDDER = None
INDEX = None
GENERATOR = None


In [15]:
# Load KB

def load_knowledge_base():
    global KB, KB_QUESTIONS, KB_ANSWERS
    with open(KB_FILE, "r", encoding="utf-8") as f:
        KB = json.load(f)
    KB_QUESTIONS = [item["question"] for item in KB]
    KB_ANSWERS = [item["answer"] for item in KB]
    print(f"Loaded {len(KB)} entries from {KB_FILE}")


In [16]:
# Build and Load FAISS Index

def build_faiss_index():
    global INDEX, EMBEDDER

    EMBEDDER = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print("Loaded embedding model:", EMBEDDING_MODEL_NAME)

    # encode questions
    question_embeddings = EMBEDDER.encode(
        KB_QUESTIONS,
        convert_to_numpy=True,
        show_progress_bar=False
    ).astype("float32")

    faiss.normalize_L2(question_embeddings)
    dim = question_embeddings.shape[1]

    INDEX = faiss.IndexFlatIP(dim)
    INDEX.add(question_embeddings)
    print("FAISS index size:", INDEX.ntotal)

    # save index if you want to persist it
    faiss.write_index(INDEX, INDEX_FILE)
    print(f"Index saved to {INDEX_FILE}")


def load_or_build_index():
    global INDEX, EMBEDDER
    EMBEDDER = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print("Loaded embedding model:", EMBEDDING_MODEL_NAME)

    if os.path.exists(INDEX_FILE):
        INDEX = faiss.read_index(INDEX_FILE)
        print(f"Loaded FAISS index from {INDEX_FILE} with {INDEX.ntotal} vectors")
    else:
        print("Index file not found, building a new one...")
        build_faiss_index()


In [17]:
## Retrieval with top K and Threshold

def retrieve_top_k(query: str, k: int = 3):
    # encode query
    query_emb = EMBEDDER.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(query_emb)

    distances, indices = INDEX.search(query_emb, k)

    results = []
    for score, idx in zip(distances[0], indices[0]):
        results.append({
            "id": KB[idx]["id"],
            "question": KB[idx]["question"],
            "answer": KB[idx]["answer"],
            "similarity": float(score)
        })
    return results


def get_best_match(query: str, k: int = 3):
    matches = retrieve_top_k(query, k=k)
    best = matches[0]
    if best["similarity"] >= SIMILARITY_THRESHOLD:
        return best
    else:
        return None   # signals “not confident”


## Load TF and Initial Gradio UI



In [18]:
def load_generator():
    global GENERATOR
    GENERATOR = pipeline(
        "text2text-generation",
        model=GENERATOR_MODEL_NAME
    )
    print("Loaded generator model:", GENERATOR_MODEL_NAME)


## Simple Generation Helper




In [19]:
def generate_response(user_query: str, kb_answer: str) -> str:
    """
    Use Flan-T5 to rewrite the KB answer in a friendly, safe way.
    """
    prompt = (
        "You are a friendly, safe heart-health education assistant. "
        "Use the provided information to answer the user's question. "
        "Do NOT diagnose, do NOT recommend specific medications or dosages, "
        "and always encourage talking to a healthcare professional.\n\n"
        f"User question: {user_query}\n"
        f"Information: {kb_answer}\n\n"
        "Answer:"
    )
    out = GENERATOR(
        prompt,
        max_length=256,
        num_return_sequences=1
    )[0]["generated_text"]
    return out.strip()

We’ll implement three intents:

wellness_faq – normal heart/medication questions → use retrieval + generator

diagnostic_safety – crisis / “what should I take for chest pain” style → safety message

general_chat – small talk or off-topic → gentle redirection + maybe generator

## Intent detection (rule-based)

In [20]:
def detect_intent(user_text: str) -> str:
    text = user_text.lower()

    # safety / diagnostic red flags
    safety_keywords = [
        "chest pain", "severe chest", "pressure in my chest",
        "can't breathe", "cant breathe", "difficulty breathing",
        "shortness of breath", "trouble breathing",
        "heart attack", "stroke",
        "kill myself", "suicidal", "overdose", "pass out", "fainting"
    ]
    if any(kw in text for kw in safety_keywords):
        return "diagnostic_safety"

    # wellness / faq-ish
    wellness_keywords = [
        "heart", "cardio", "blood pressure", "hypertension",
        "cholesterol", "statin", "beta blocker", "ace inhibitor",
        "arb", "atrial fibrillation", "afib", "angina",
        "heart failure", "cardiac", "aspirin", "blood thinner"
    ]
    if any(kw in text for kw in wellness_keywords):
        return "wellness_faq"

    # default
    return "general_chat"


## Safety Response

In [21]:
def safety_response(user_text: str) -> str:
    return (
        "I'm really glad you reached out. I’m not a medical professional and "
        "cannot give diagnosis or emergency advice. If you are having chest pain, "
        "trouble breathing, feel like you might pass out, or think you might be having "
        "a heart attack or stroke, please call 911 or your local emergency number "
        "immediately, or seek urgent medical care in person.\n\n"
        "For non-emergency concerns about your heart or medications, please contact your "
        "doctor, cardiologist, or pharmacist as soon as you can."
    )


## General chat response (fallback to T5)

In [22]:
def general_chat_response(user_text: str) -> str:
    """
    Use T5 to give a lightweight, safe reply, keeping focus on heart health.
    """
    prompt = (
        "You are a friendly heart-health education assistant. "
        "You mainly answer questions about heart health, blood pressure, "
        "and medications in a general, non-diagnostic way.\n\n"
        "User: " + user_text + "\n"
        "Assistant:"
    )
    out = GENERATOR(prompt, max_length=128, num_return_sequences=1)[0]["generated_text"]
    return out.strip()


## Main respond function

In [23]:
def respond(user_text: str) -> str:
    intent = detect_intent(user_text)

    # 1. Safety first
    if intent == "diagnostic_safety":
        return safety_response(user_text)

    # 2. FAQ: use retrieval + generator
    if intent == "wellness_faq":
        best = get_best_match(user_text, k=3)
        if best is not None:
            kb_answer = best["answer"]
            try:
                final_answer = generate_response(user_text, kb_answer)
            except Exception:
                # if generator fails for some reason, just return KB answer
                final_answer = kb_answer

            # Optionally add a short disclaimer
            final_answer += (
                "\n\nDisclaimer: This is general heart-health information and "
                "cannot replace advice from your own healthcare professional."
            )
            return final_answer
        else:
            # No confident match in the KB
            return (
                "I'm not completely sure based on the information I have. "
                "Your question may be more specific than my current heart-health FAQs. "
                "Please talk with your doctor or pharmacist for personalized guidance."
            )

    # 3. General chat – use T5 but keep it safe and on-topic-ish
    return general_chat_response(user_text)


In [31]:
# CODE TO VERIFY/CORRECT IN YOUR FILE

def detect_intent(user_input):
    """Classifies the user's intent and checks for KB match."""
    text = user_input.lower()

    # 1. Safety Block (Diagnostic Safety) - THIS MUST BE CHECKED FIRST
    unsafe_keywords = ["i have", "symptom", "pain", "cure", "medicine", "pill", "doctor"]
    if any(word in text for word in unsafe_keywords):
        # Return the intent name and NO knowledge base data
        return "diagnostic_safety", None

    # 2. Wellness FAQ (Retrieval Check)
    # The search_knowledge_base function should return a dictionary if it finds a match.
    kb_match = search_knowledge_base(user_input)
    if kb_match:
        return "wellness_faq", kb_match

    # 3. Greetings
    if any(word in text for word in ["hi", "hello", "hey", "greetings"]):
        return "greet", None

    # 4. General Chat (Fallback)
    return "general_chat", None

## Fix Response Routing

In [41]:
# CODE TO VERIFY/CORRECT IN YOUR FILE

def respond(message): # Or chat_response(message, history) if using a history argument
    """The main response function running the full pipeline."""

    # Step 1: Detect Intent
    intent, kb_data = detect_intent(message) # This gets the intent and the KB data (if any)

    # Step 2: Route Response based on Intent (ORDER MATTERS HERE!)

    if intent == "diagnostic_safety":
        # THIS MUST RETURN THE SAFETY MESSAGE
        return "I am an AI wellness assistant, not a doctor. I cannot provide medical advice, diagnoses, or treatment suggestions. Please consult a healthcare professional."

    elif intent == "wellness_faq":
        # RAG Success: Return the precise answer from your KB
        return f"{kb_data['answer']} (Source: Knowledge Base)"

    elif intent == "greet":
        return "Hello! I am your wellness assistant. Feel free to ask me about nutrition, exercise, or general health terms."

    elif intent == "general_chat":
        # T5 Fallback: Use the T5 model for general conversation
        # Note: You should be prompting T5 for a general response, not using a prompt that might lead to an unintended response like "Is there anything I can help you with?"
        prompt = f"Answer this general question briefly: {message}"
        response = GENERATOR(prompt, max_length=128, do_sample=False)[0]['generated_text']
        return response

    return "An unexpected error occurred."

## Refine T5 Fallback prompt

In [45]:
# CODE TO VERIFY/CORRECT IN YOUR FILE
elif intent == "general_chat":
    # T5 Fallback: Use the T5 model for general conversation
    # Use a strong prompt to force a specific, non-echoing answer
    prompt = f"Answer this general question briefly in one or two sentences: {message}"
    response = GENERATOR(prompt, max_length=128, do_sample=False)[0]['generated_text']
    return response

SyntaxError: invalid syntax (ipython-input-4261022796.py, line 2)

## Gradio UI (basic running app)

In [46]:
# Assuming your main script is named app.py
%cd /content/Healthcare-Chatbot
!python3 app.py

/content/Healthcare-Chatbot


In [47]:
def chat_fn(user_message, history):
    """
    Gradio-compatible chat function.
    history: list of [user, bot] pairs
    """
    bot_reply = respond(user_message)
    history = history or []
    history.append((user_message, bot_reply))
    return history, history


def main():
    # Init everything once
    load_knowledge_base()
    load_or_build_index()
    load_generator()

    with gr.Blocks() as demo:
        gr.Markdown("# ❤️ Heart Health Chatbot\n_A safe, educational assistant about heart health and medications._")

        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Ask a question about heart health or medications")
        clear = gr.Button("Clear")

        msg.submit(chat_fn, [msg, chatbot], [chatbot, msg])
        clear.click(lambda: [], None, chatbot)

    demo.launch()


if __name__ == "__main__":
    main()


Loaded 50 entries from healthcare_data.json
Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2
Loaded FAISS index from faq.index with 50 vectors


Device set to use cpu


Loaded generator model: google/flan-t5-base


/tmp/ipython-input-362707968.py:21: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67dee346a6810652a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Finalize Core Logic again to improve processes

In [25]:
# Assuming all imports (json, os, faiss, SentenceTransformer, Gradio, pipeline)
# and model loading (KB, INDEX, EMBEDDER, GENERATOR) are done at the top of the file.

# ----------------------------------------------------------------------
# A. Retrieval Logic (Nov 17)
# ----------------------------------------------------------------------
# NOTE: Ensure this function is present in your .py file!

def search_knowledge_base(query, top_k=1, threshold=0.75):
    """Searches the FAISS index for the top_k most similar questions."""

    # Check if RAG components are loaded
    if INDEX is None or EMBEDDER is None:
        print("ERROR: RAG components not loaded.")
        return None

    # Encode the user query, normalize, and search
    query_embedding = EMBEDDER.encode([query], convert_to_tensor=True).cpu().numpy()
    faiss.normalize_L2(query_embedding)
    D, I = INDEX.search(query_embedding, top_k)

    best_score = D[0][0]
    best_idx = I[0][0]

    # The threshold is now set higher (0.75) for high-confidence match
    if best_score <= threshold: # Note: FAISS uses L2 distance, so lower score is better (distance, not similarity)
        # If the distance is too high (score > 0.75), return None
        return None

    # Return the full Q&A entry from your knowledge base
    return KB[best_idx]


# ----------------------------------------------------------------------
# B. Dialogue Flow & Intent Router (Nov 19)
# ----------------------------------------------------------------------

def detect_intent(user_input):
    """Classifies the user's intent and checks for KB match."""
    text = user_input.lower()

    # 1. Safety Block (Diagnostic Safety)
    unsafe_keywords = ["i have", "symptom", "pain", "cure", "medicine", "pill", "doctor"]
    if any(word in text for word in unsafe_keywords):
        return "diagnostic_safety", None

    # 2. Wellness FAQ (Retrieval Check)
    kb_match = search_knowledge_base(user_input)
    if kb_match:
        return "wellness_faq", kb_match

    # 3. Greetings
    if any(word in text for word in ["hi", "hello", "hey", "greetings"]):
        return "greet", None

    # 4. General Chat (Fallback)
    return "general_chat", None


# ----------------------------------------------------------------------
# C. Integrate All Components (Nov 20)
# ----------------------------------------------------------------------

def chat_response(message, history):
    """The main response function running the full pipeline."""

    # Step 1: Detect Intent
    intent, kb_data = detect_intent(message)

    # Step 2: Route Response based on Intent
    if intent == "diagnostic_safety":
        return "I am an AI wellness assistant, not a doctor. I cannot provide medical advice, diagnoses, or treatment suggestions. Please consult a healthcare professional."

    elif intent == "greet":
        return "Hello! I am your wellness assistant. Feel free to ask me about nutrition, exercise, or general health terms."

    elif intent == "wellness_faq":
        # RAG Success: Return the precise answer from your KB
        return f"{kb_data['answer']} (Source: Knowledge Base)"

    elif intent == "general_chat":
        # T5 Fallback: Use the T5 model for general conversation
        prompt = f"Answer this general question briefly: {message}"
        response = GENERATOR(prompt, max_length=128, do_sample=False)[0]['generated_text']
        return response

    return "An unexpected error occurred."

In [27]:
%cd /content/Healthcare-Chatbot
!python3 Ryan_Nguyen_Chatbotpt2

/content/Healthcare-Chatbot
python3: can't open file '/content/Healthcare-Chatbot/Ryan_Nguyen_Chatbotpt2': [Errno 2] No such file or directory


In [28]:
%cd /content/Healthcare-Chatbot
!python app.py


/content/Healthcare-Chatbot


## Stage File

In [51]:
!pwd

/content/Healthcare-Chatbot


In [52]:
!ls

app.py	faq.index  healthcare_data.json  README.md  ryan_nguyen_chatbot_py.py


In [56]:
!git add Ryan_Nguyen_Chatbotpt2.ipynb

fatal: pathspec 'Ryan_Nguyen_Chatbotpt2.ipynb' did not match any files


In [57]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


GridBox(children=(Dropdown(layout=Layout(width='auto'), options=('google/gemini-2.5-flash', 'google/gemini-2.5…